In [127]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('Chinook.db')
cursor = conn.execute("SELECT * FROM Genre LIMIT 5")
rows = cursor.fetchall()

rows

[(1, 'Rock'),
 (2, 'Jazz'),
 (3, 'Metal'),
 (4, 'Alternative & Punk'),
 (5, 'Rock And Roll')]

In [112]:
data = ''
with open('Chinook_db_description.txt','r') as file:
    data= file.readlines()

text = " ".join(data)
#print(text)


In [113]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.agents import load_tools
from langchain import hub
from langchain.tools import tool
from langchain_experimental.tools.python.tool import PythonREPLTool

load_dotenv()

True

In [114]:
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [115]:
llm = ChatOpenAI(temperature=0,model='gpt-3.5-turbo-0301')

In [116]:
@tool("search", return_direct=True)
def search(query :str) -> str:
    """Look up things"""
    return "Langchain"

@tool
def multiply(a :int,b:int) -> int:
    """multiply two numbers"""
    return a * b

@tool
def get_word_length(word :str)-> int:
    """Returns the length of the word"""
    return len(word)

# @tool
# def get_data_from_db(query :str):
#     """Get data from my database"""
    
#     return pd.DataFrame({'A':[1,2,3], 'B':[8,9,10]})


In [117]:
#tools = load_tools(['llm-math'],llm=llm)
tools = [get_data_from_db,PythonREPLTool()]
tools

[StructuredTool(name='get_data_from_db', description='get_data_from_db(sql_query) - Given a SQL query get the data from the databse as a dataframe', args_schema=<class 'pydantic.v1.main.get_data_from_dbSchema'>, func=<function get_data_from_db at 0x000002504A1CF1C0>),
 PythonREPLTool()]

In [118]:
from langchain.agents import create_react_agent, AgentExecutor
import langchain


In [119]:
langchain.debug= False
agent = create_react_agent(llm, tools,prompt)
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor.invoke({"input":"Create a scatter plot of the data from my database"})
langchain.debug= False



> Entering new AgentExecutor chain...
I need to retrieve the data from the database and then use a plotting library to create the scatter plot.
Action: 
- get_data_from_db
- SELECT x_column, y_column FROM my_table
Action Input: sql_query = "SELECT x_column, y_column FROM my_table"- get_data_from_db
- SELECT x_column, y_column FROM my_table is not a valid tool, try one of [get_data_from_db, Python_REPL].I need to use get_data_from_db to retrieve the data and then use a plotting library to create the scatter plot.
Action: 
- get_data_from_db
- sql_query = "SELECT x_column, y_column FROM my_table"
- import matplotlib.pyplot as plt
- plt.scatter(data['x_column'], data['y_column'])
Action Input: 
- sql_query = "SELECT x_column, y_column FROM my_table"
- None- get_data_from_db
- sql_query = "SELECT x_column, y_column FROM my_table"
- import matplotlib.pyplot as plt
- plt.scatter(data['x_column'], data['y_column']) is not a valid tool, try one of [get_data_from_db, Python_REPL].I need to us

In [120]:

def get_db_description():
    """Get the schema, table names, column names and the relationships of the tables in the database
    """
    data = ''
    try:
        with open('Chinook_db_description.txt','r') as file:
            data= file.readlines()

        text = " ".join(data)
        return text
    except Exception as e:
        print("No db description file found")
        return None

@tool  
def get_data_from_db(sql_query):
    """
    Given a SQL query get the data from the databse as a dataframe
    """
    print("getting data from db")
    try:
        conn = sqlite3.connect('Chinook.db')
        df = pd.read_sql(sql_query,conn)
        #print(df.head())
        df.to_csv('data.csv',index=False)
        conn.close()
        return df
    except Exception as e:
        #st.error(e)
        conn.close()
        return None



In [121]:
#tools = load_tools(['llm-math'],llm=llm)
tools = [get_data_from_db,PythonREPLTool()]
tools

[StructuredTool(name='get_data_from_db', description='get_data_from_db(sql_query) - Given a SQL query get the data from the databse as a dataframe', args_schema=<class 'pydantic.v1.main.get_data_from_dbSchema'>, func=<function get_data_from_db at 0x0000025047D49240>),
 PythonREPLTool()]

In [122]:
get_db_description()

"The Chinook database is a sample database used for learning and practicing SQL. It mimics a digital media store, including tables for artists, albums, media tracks, invoices, customers, and employees. Here's an overview of the schema:\n \n Tables and their columns:\n Artist\n \n ArtistId (INTEGER, Primary Key)\n Name (NVARCHAR)\n Album\n \n AlbumId (INTEGER, Primary Key)\n Title (NVARCHAR)\n ArtistId (INTEGER, Foreign Key referencing Artist)\n Track\n \n TrackId (INTEGER, Primary Key)\n Name (NVARCHAR)\n AlbumId (INTEGER, Foreign Key referencing Album)\n MediaTypeId (INTEGER, Foreign Key referencing MediaType)\n GenreId (INTEGER, Foreign Key referencing Genre)\n Composer (NVARCHAR)\n Milliseconds (INTEGER)\n Bytes (INTEGER)\n UnitPrice (NUMERIC)\n MediaType\n \n MediaTypeId (INTEGER, Primary Key)\n Name (NVARCHAR)\n Genre\n \n GenreId (INTEGER, Primary Key)\n Name (NVARCHAR)\n Invoice\n \n InvoiceId (INTEGER, Primary Key)\n CustomerId (INTEGER, Foreign Key referencing Customer)\n Invo

In [ ]:
def create_run_Python_agent():
    tools = [PythonREPLTool()]

In [130]:
#user_query = "Create a heatmap of the popularity of the genre by country" + ". Here's the description of the database" + get_db_description() +"Use the viridis colormap in any plots. The plot size shoule be 10,6"
#user_query = "Who are the top 10 artists by sales" + ". Here's the description of the database" + get_db_description() +"Use the viridis colormap in any plots. The plot size shoule be 10,6"
user_query = "Create a bar plot of the top 10 artists by sales" + ". Here's the description of the database" + get_db_description() +"Use the viridis colormap in any plots. The plot size shoule be 10,6"
#user_query = "What are the columns in the artist table" + "Here's the description of the database" + get_db_description()

langchain.debug= True
agent = create_react_agent(llm, tools,prompt)
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=False,handle_parsing_errors=False)

agent_executor.invoke({"input":user_query})
langchain.debug= False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Create a bar plot of the top 10 artists by sales. Here's the description of the databaseThe Chinook database is a sample database used for learning and practicing SQL. It mimics a digital media store, including tables for artists, albums, media tracks, invoices, customers, and employees. Here's an overview of the schema:\n \n Tables and their columns:\n Artist\n \n ArtistId (INTEGER, Primary Key)\n Name (NVARCHAR)\n Album\n \n AlbumId (INTEGER, Primary Key)\n Title (NVARCHAR)\n ArtistId (INTEGER, Foreign Key referencing Artist)\n Track\n \n TrackId (INTEGER, Primary Key)\n Name (NVARCHAR)\n AlbumId (INTEGER, Foreign Key referencing Album)\n MediaTypeId (INTEGER, Foreign Key referencing MediaType)\n GenreId (INTEGER, Foreign Key referencing Genre)\n Composer (NVARCHAR)\n Milliseconds (INTEGER)\n Bytes (INTEGER)\n UnitPrice (NUMERIC)\n MediaType\n \n MediaTypeId (INTEGER, Primary Key)\n Name (NVARCHAR)\n Gen

Error in ConsoleCallbackHandler.on_tool_end callback: AttributeError("'NoneType' object has no attribute 'strip'")


[llm/end] [chain:AgentExecutor > chain:RunnableSequence > llm:ChatOpenAI] [1.72s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "We need to get the data of top 10 artists by sales from the database and create a bar plot using matplotlib.\nAction: get_data_from_db\nAction Input: \n```\nSELECT \n    Artist.Name AS ArtistName, \n    SUM(InvoiceLine.Quantity) AS TotalSales\nFROM \n    Artist \n    INNER JOIN Album ON Artist.ArtistId = Album.ArtistId \n    INNER JOIN Track ON Album.AlbumId = Track.AlbumId \n    INNER JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId \nGROUP BY \n    Artist.ArtistId \nORDER BY \n    TotalSales DESC \nLIMIT 10;\n```",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGenerationChunk",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessageChunk"
   

Error in ConsoleCallbackHandler.on_tool_end callback: AttributeError("'NoneType' object has no attribute 'strip'")


[llm/end] [chain:AgentExecutor > chain:RunnableSequence > llm:ChatOpenAI] [1.62s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "We need to store the data obtained from the database in a pandas dataframe.\nAction: get_data_from_db\nAction Input: \n```\nSELECT \n    Artist.Name AS ArtistName, \n    SUM(InvoiceLine.Quantity) AS TotalSales\nFROM \n    Artist \n    INNER JOIN Album ON Artist.ArtistId = Album.ArtistId \n    INNER JOIN Track ON Album.AlbumId = Track.AlbumId \n    INNER JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId \nGROUP BY \n    Artist.ArtistId \nORDER BY \n    TotalSales DESC \nLIMIT 10;\n```",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGenerationChunk",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessageChunk"
          ],
          "kwargs": {
 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The database is not connected or the schema is incorrect. We need to check the database connection details and make sure the schema is correct.
Action: Check the database connection details and schema.`